In [ ]:
## This file implements neural networks and logistic regression on pyopresabsSTCC_quant.
## Due to the imbalanced data, we implement the over-sampling method and the combination of 
## over- and under-sampling method, in case of imbalanced data.
## For fully-connected neural networks, the accuracy is 84.87% for combination data and 81.12% for over-sampling data.
## To improve neural network performance, we implement lasso regression to select important features.
## However, with the new dataset, the accuracy becomes 78.57% for combination, and 79.02% for over-sampling data.
## For logistic regression, the accuracy is 98.32% for combination data and 99.3% for over-sampling data.
## For random forest, the accuracy is 98.32% for combination, and 100% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 99.27% for combination data, and 98.79% for over-sampling.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/pyopresabsSTCC_quant.csv')
df.shape

(255, 541)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0.158375
1      0.157875
2      0.310875
3      0.160125
4      0.160500
5      0.205000
6      0.158250
7      0.158375
8      0.166000
9      0.157750
10     0.159375
11     0.159000
12     0.161000
13     0.161875
14     0.157875
15     0.157125
16     0.305000
17     0.281250
18     0.244625
19     0.156875
20     0.169625
21     0.282500
22     0.156750
23     0.161875
24     0.525750
25     0.155375
26     0.171375
27     0.156500
28     0.158625
29     0.288500
         ...   
225    0.179000
226    0.333889
227    0.400222
228    0.499444
229    0.523000
230    0.267250
231    0.209000
232    0.243625
233    0.201875
234    0.160000
235    0.184250
236    0.178000
237    0.157000
238    0.158250
239    0.201000
240    0.232750
241    0.174250
242    0.159250
243    0.160125
244    0.154000
245    0.156875
246    0.233375
247    0.157375
248    0.158375
249    0.158625
250    0.231250
251    0.174625
252    0.157000
253    0.167625
254    0.163000
Name: pheno, Length: 255

In [5]:
df.head()

,id,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,...,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,pheno,ST,CC
0,107,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0.158375,5,5
1,109,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0.157875,8,8
2,115,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0.310875,5,5
3,120335,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0.160125,5,5
4,120337,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0.160500,5,5


In [6]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [7]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [8]:
df['pheno']
df['pheno'].value_counts()

0    237
1     18
Name: pheno, dtype: int64

In [9]:
df.shape

(255, 541)

In [10]:
df_clean = df.drop(columns=['id'])

In [11]:
df_clean.shape

(255, 540)

In [12]:
df_clean.head()

,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGT,...,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,pheno,ST,CC
0,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,5,5
1,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,8,8
2,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,5,5
3,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,5,5
4,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,5,5


In [13]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 539) (255,)


In [14]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 192), (1, 202)]


Using TensorFlow backend.


In [15]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 237), (1, 237)]


In [16]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [17]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [18]:
############# Fully-Connected Neural Network ################

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [20]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [21]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 275 samples, validate on 119 samples
Epoch 1/1000
275/275 [==============================] - 0s 449us/step - loss: 3.7530 - accuracy: 0.5382 - val_loss: 0.6327 - val_accuracy: 0.4874
Epoch 2/1000
275/275 [==============================] - 0s 84us/step - loss: 0.6380 - accuracy: 0.5273 - val_loss: 0.6188 - val_accuracy: 0.6555
Epoch 3/1000
275/275 [==============================] - 0s 109us/step - loss: 0.6207 - accuracy: 0.6073 - val_loss: 0.6048 - val_accuracy: 0.6807
Epoch 4/1000
275/275 [==============================] - 0s 99us/step - loss: 0.6093 - accuracy: 0.6436 - val_loss: 0.5961 - val_accuracy: 0.7227
Epoch 5/1000
275/275 [==============================] - 0s 106us/step - loss: 0.6054 - accuracy: 0.6691 - val_loss: 0.5864 - val_accuracy: 0.7479
Epoch 6/1000
275/275 [==============================] - 0s 115us/step - loss: 0.5957 - accuracy: 0.6945 - val_loss: 0.5780 - val_accuracy: 0.7395
Epoch 7/1000
275/275 [==============================] - 0s 124us/step - loss: 0.

275/275 [==============================] - 0s 125us/step - loss: 0.4035 - accuracy: 0.8582 - val_loss: 0.3823 - val_accuracy: 0.8739
Epoch 57/1000
275/275 [==============================] - 0s 114us/step - loss: 0.3958 - accuracy: 0.8400 - val_loss: 0.3913 - val_accuracy: 0.8487
Epoch 58/1000
275/275 [==============================] - 0s 112us/step - loss: 0.3959 - accuracy: 0.8400 - val_loss: 0.3962 - val_accuracy: 0.8319
Epoch 59/1000
275/275 [==============================] - 0s 104us/step - loss: 0.4093 - accuracy: 0.8182 - val_loss: 0.3798 - val_accuracy: 0.8487
Epoch 60/1000
275/275 [==============================] - 0s 83us/step - loss: 0.3910 - accuracy: 0.8436 - val_loss: 0.3752 - val_accuracy: 0.8908
Epoch 61/1000
275/275 [==============================] - 0s 95us/step - loss: 0.3894 - accuracy: 0.8473 - val_loss: 0.3621 - val_accuracy: 0.9160
Epoch 62/1000
275/275 [==============================] - 0s 92us/step - loss: 0.4000 - accuracy: 0.8364 - val_loss: 0.3726 - val_accur

Epoch 112/1000
275/275 [==============================] - 0s 111us/step - loss: 0.2986 - accuracy: 0.8582 - val_loss: 0.3138 - val_accuracy: 0.8571
Epoch 113/1000
275/275 [==============================] - 0s 112us/step - loss: 0.3153 - accuracy: 0.8436 - val_loss: 0.2903 - val_accuracy: 0.9160
Epoch 114/1000
275/275 [==============================] - 0s 116us/step - loss: 0.3545 - accuracy: 0.8291 - val_loss: 0.2884 - val_accuracy: 0.8992
Epoch 115/1000
275/275 [==============================] - 0s 93us/step - loss: 0.2922 - accuracy: 0.8582 - val_loss: 0.3137 - val_accuracy: 0.8571
Epoch 116/1000
275/275 [==============================] - 0s 102us/step - loss: 0.2902 - accuracy: 0.8582 - val_loss: 0.2961 - val_accuracy: 0.8739
Epoch 117/1000
275/275 [==============================] - 0s 124us/step - loss: 0.3026 - accuracy: 0.8473 - val_loss: 0.3123 - val_accuracy: 0.8908
Epoch 118/1000
275/275 [==============================] - 0s 135us/step - loss: 0.3165 - accuracy: 0.8327 - val_l

275/275 [==============================] - 0s 86us/step - loss: 0.2538 - accuracy: 0.8509 - val_loss: 0.2850 - val_accuracy: 0.8571
Epoch 168/1000
275/275 [==============================] - 0s 84us/step - loss: 0.2585 - accuracy: 0.8618 - val_loss: 0.2821 - val_accuracy: 0.8571
Epoch 169/1000
275/275 [==============================] - 0s 89us/step - loss: 0.2629 - accuracy: 0.8582 - val_loss: 0.2533 - val_accuracy: 0.8824
Epoch 170/1000
275/275 [==============================] - 0s 88us/step - loss: 0.2775 - accuracy: 0.8691 - val_loss: 0.2588 - val_accuracy: 0.9160
Epoch 171/1000
275/275 [==============================] - 0s 84us/step - loss: 0.2493 - accuracy: 0.8582 - val_loss: 0.2482 - val_accuracy: 0.9160
Epoch 172/1000
275/275 [==============================] - 0s 89us/step - loss: 0.2675 - accuracy: 0.8364 - val_loss: 0.2714 - val_accuracy: 0.8571
Epoch 173/1000
275/275 [==============================] - 0s 82us/step - loss: 0.2516 - accuracy: 0.8655 - val_loss: 0.2825 - val_acc

Epoch 223/1000
275/275 [==============================] - 0s 87us/step - loss: 0.2317 - accuracy: 0.8473 - val_loss: 0.2586 - val_accuracy: 0.8487
Epoch 224/1000
275/275 [==============================] - 0s 83us/step - loss: 0.2545 - accuracy: 0.8436 - val_loss: 0.2453 - val_accuracy: 0.8739
Epoch 225/1000
275/275 [==============================] - 0s 88us/step - loss: 0.2545 - accuracy: 0.8436 - val_loss: 0.2508 - val_accuracy: 0.8739
Epoch 226/1000
275/275 [==============================] - 0s 86us/step - loss: 0.2425 - accuracy: 0.8691 - val_loss: 0.2404 - val_accuracy: 0.8739
Epoch 227/1000
275/275 [==============================] - 0s 87us/step - loss: 0.2275 - accuracy: 0.8691 - val_loss: 0.2629 - val_accuracy: 0.8487
Epoch 228/1000
275/275 [==============================] - 0s 91us/step - loss: 0.2512 - accuracy: 0.8400 - val_loss: 0.2659 - val_accuracy: 0.8487
Epoch 229/1000
275/275 [==============================] - 0s 87us/step - loss: 0.2388 - accuracy: 0.8618 - val_loss: 0

...

275/275 [==============================] - 0s 92us/step - loss: 0.2312 - accuracy: 0.8582 - val_loss: 0.2495 - val_accuracy: 0.8571
Epoch 781/1000
275/275 [==============================] - 0s 103us/step - loss: 0.2311 - accuracy: 0.8582 - val_loss: 0.2516 - val_accuracy: 0.8571
Epoch 782/1000
275/275 [==============================] - 0s 133us/step - loss: 0.2361 - accuracy: 0.8582 - val_loss: 0.2578 - val_accuracy: 0.8571
Epoch 783/1000
275/275 [==============================] - 0s 119us/step - loss: 0.2325 - accuracy: 0.8582 - val_loss: 0.2505 - val_accuracy: 0.8571
Epoch 784/1000
275/275 [==============================] - 0s 103us/step - loss: 0.2325 - accuracy: 0.8582 - val_loss: 0.2517 - val_accuracy: 0.8571
Epoch 785/1000
275/275 [==============================] - 0s 91us/step - loss: 0.2323 - accuracy: 0.8582 - val_loss: 0.2501 - val_accuracy: 0.8571
Epoch 786/1000
275/275 [==============================] - 0s 88us/step - loss: 0.2318 - accuracy: 0.8582 - val_loss: 0.2521 - val

275/275 [==============================] - 0s 95us/step - loss: 0.2323 - accuracy: 0.8582 - val_loss: 0.2520 - val_accuracy: 0.8571
Epoch 836/1000
275/275 [==============================] - 0s 91us/step - loss: 0.2320 - accuracy: 0.8582 - val_loss: 0.2551 - val_accuracy: 0.8571
Epoch 837/1000
275/275 [==============================] - 0s 97us/step - loss: 0.2320 - accuracy: 0.8582 - val_loss: 0.2533 - val_accuracy: 0.8571
Epoch 838/1000
275/275 [==============================] - ETA: 0s - loss: 0.2581 - accuracy: 0.81 - 0s 107us/step - loss: 0.2325 - accuracy: 0.8582 - val_loss: 0.2532 - val_accuracy: 0.8571
Epoch 839/1000
275/275 [==============================] - 0s 113us/step - loss: 0.2322 - accuracy: 0.8582 - val_loss: 0.2524 - val_accuracy: 0.8571
Epoch 840/1000
275/275 [==============================] - 0s 102us/step - loss: 0.2315 - accuracy: 0.8582 - val_loss: 0.2489 - val_accuracy: 0.8571
Epoch 841/1000
275/275 [==============================] - 0s 93us/step - loss: 0.2321 - 

275/275 [==============================] - 0s 131us/step - loss: 0.2307 - accuracy: 0.8582 - val_loss: 0.2520 - val_accuracy: 0.8571
Epoch 891/1000
275/275 [==============================] - 0s 113us/step - loss: 0.2303 - accuracy: 0.8582 - val_loss: 0.2546 - val_accuracy: 0.8571
Epoch 892/1000
275/275 [==============================] - 0s 99us/step - loss: 0.2305 - accuracy: 0.8582 - val_loss: 0.2496 - val_accuracy: 0.8571
Epoch 893/1000
275/275 [==============================] - 0s 102us/step - loss: 0.2306 - accuracy: 0.8582 - val_loss: 0.2487 - val_accuracy: 0.8571
Epoch 894/1000
275/275 [==============================] - 0s 104us/step - loss: 0.2315 - accuracy: 0.8582 - val_loss: 0.2492 - val_accuracy: 0.8571
Epoch 895/1000
275/275 [==============================] - 0s 164us/step - loss: 0.2333 - accuracy: 0.8582 - val_loss: 0.2558 - val_accuracy: 0.8571
Epoch 896/1000
275/275 [==============================] - 0s 146us/step - loss: 0.2308 - accuracy: 0.8582 - val_loss: 0.2503 - v

275/275 [==============================] - 0s 84us/step - loss: 0.2299 - accuracy: 0.8582 - val_loss: 0.2501 - val_accuracy: 0.8571
Epoch 946/1000
275/275 [==============================] - 0s 96us/step - loss: 0.2299 - accuracy: 0.8582 - val_loss: 0.2507 - val_accuracy: 0.8571
Epoch 947/1000
275/275 [==============================] - 0s 86us/step - loss: 0.2297 - accuracy: 0.8582 - val_loss: 0.2483 - val_accuracy: 0.8487
Epoch 948/1000
275/275 [==============================] - 0s 103us/step - loss: 0.2297 - accuracy: 0.8582 - val_loss: 0.2482 - val_accuracy: 0.8487
Epoch 949/1000
275/275 [==============================] - 0s 121us/step - loss: 0.2301 - accuracy: 0.8582 - val_loss: 0.2492 - val_accuracy: 0.8487
Epoch 950/1000
275/275 [==============================] - 0s 106us/step - loss: 0.2296 - accuracy: 0.8582 - val_loss: 0.2525 - val_accuracy: 0.8487
Epoch 951/1000
275/275 [==============================] - 0s 96us/step - loss: 0.2300 - accuracy: 0.8582 - val_loss: 0.2495 - val_

275/275 [==============================] - 0s 95us/step - loss: 0.2290 - accuracy: 0.8582 - val_loss: 0.2482 - val_accuracy: 0.8487


In [23]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

119/119 [==============================] - 0s 73us/step
combination test accuracy: 84.87%


In [37]:
#### improve neural network
model1_comb3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [38]:
model1_comb3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [39]:
model1_comb3.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 275 samples, validate on 119 samples
Epoch 1/1000
275/275 [==============================] - 0s 524us/step - loss: 5.8240 - accuracy: 0.5818 - val_loss: 0.6549 - val_accuracy: 0.5042
Epoch 2/1000
275/275 [==============================] - 0s 174us/step - loss: 0.6831 - accuracy: 0.5455 - val_loss: 0.6494 - val_accuracy: 0.6639
Epoch 3/1000
275/275 [==============================] - 0s 186us/step - loss: 0.6513 - accuracy: 0.5964 - val_loss: 0.6340 - val_accuracy: 0.6807
Epoch 4/1000
275/275 [==============================] - 0s 190us/step - loss: 0.6457 - accuracy: 0.6109 - val_loss: 0.6185 - val_accuracy: 0.5882
Epoch 5/1000
275/275 [==============================] - 0s 175us/step - loss: 0.6418 - accuracy: 0.6000 - val_loss: 0.6136 - val_accuracy: 0.5462
Epoch 6/1000
275/275 [==============================] - 0s 194us/step - loss: 0.6304 - accuracy: 0.5927 - val_loss: 0.5949 - val_accuracy: 0.7563
Epoch 7/1000
275/275 [==============================] - 0s 187us/step - loss: 

275/275 [==============================] - 0s 171us/step - loss: 0.3660 - accuracy: 0.8509 - val_loss: 0.3634 - val_accuracy: 0.8487
Epoch 57/1000
275/275 [==============================] - 0s 173us/step - loss: 0.3710 - accuracy: 0.8327 - val_loss: 0.3593 - val_accuracy: 0.8487
Epoch 58/1000
275/275 [==============================] - 0s 155us/step - loss: 0.3576 - accuracy: 0.8473 - val_loss: 0.3561 - val_accuracy: 0.8487
Epoch 59/1000
275/275 [==============================] - 0s 136us/step - loss: 0.3802 - accuracy: 0.8291 - val_loss: 0.3577 - val_accuracy: 0.8487
Epoch 60/1000
275/275 [==============================] - 0s 148us/step - loss: 0.3713 - accuracy: 0.8364 - val_loss: 0.3536 - val_accuracy: 0.8487
Epoch 61/1000
275/275 [==============================] - 0s 142us/step - loss: 0.3773 - accuracy: 0.8255 - val_loss: 0.3541 - val_accuracy: 0.8487
Epoch 62/1000
275/275 [==============================] - 0s 147us/step - loss: 0.3621 - accuracy: 0.8400 - val_loss: 0.3546 - val_ac

Epoch 112/1000
275/275 [==============================] - 0s 141us/step - loss: 0.3129 - accuracy: 0.8473 - val_loss: 0.3151 - val_accuracy: 0.8487
Epoch 113/1000
275/275 [==============================] - 0s 152us/step - loss: 0.3237 - accuracy: 0.8327 - val_loss: 0.3090 - val_accuracy: 0.8487
Epoch 114/1000
275/275 [==============================] - 0s 139us/step - loss: 0.3080 - accuracy: 0.8545 - val_loss: 0.3194 - val_accuracy: 0.8319
Epoch 115/1000
275/275 [==============================] - 0s 144us/step - loss: 0.3263 - accuracy: 0.8364 - val_loss: 0.3140 - val_accuracy: 0.8487
Epoch 116/1000
275/275 [==============================] - 0s 134us/step - loss: 0.3365 - accuracy: 0.8182 - val_loss: 0.3143 - val_accuracy: 0.8487
Epoch 117/1000
275/275 [==============================] - 0s 135us/step - loss: 0.3154 - accuracy: 0.8473 - val_loss: 0.3084 - val_accuracy: 0.8487
Epoch 118/1000
275/275 [==============================] - 0s 151us/step - loss: 0.3037 - accuracy: 0.8509 - val_

275/275 [==============================] - 0s 136us/step - loss: 0.4331 - accuracy: 0.7600 - val_loss: 0.3894 - val_accuracy: 0.8151
Epoch 168/1000
275/275 [==============================] - 0s 154us/step - loss: 0.4318 - accuracy: 0.7600 - val_loss: 0.3881 - val_accuracy: 0.8151
Epoch 169/1000
275/275 [==============================] - 0s 138us/step - loss: 0.4309 - accuracy: 0.7600 - val_loss: 0.3868 - val_accuracy: 0.8151
Epoch 170/1000
275/275 [==============================] - 0s 133us/step - loss: 0.4302 - accuracy: 0.7600 - val_loss: 0.3861 - val_accuracy: 0.8151
Epoch 171/1000
275/275 [==============================] - 0s 152us/step - loss: 0.4294 - accuracy: 0.7600 - val_loss: 0.3853 - val_accuracy: 0.8151
Epoch 172/1000
275/275 [==============================] - 0s 131us/step - loss: 0.4283 - accuracy: 0.7600 - val_loss: 0.3846 - val_accuracy: 0.8151
Epoch 173/1000
275/275 [==============================] - 0s 143us/step - loss: 0.4273 - accuracy: 0.7600 - val_loss: 0.3839 - 

275/275 [==============================] - 0s 148us/step - loss: 0.4001 - accuracy: 0.7600 - val_loss: 0.3592 - val_accuracy: 0.8151
Epoch 223/1000
275/275 [==============================] - 0s 142us/step - loss: 0.3998 - accuracy: 0.7600 - val_loss: 0.3586 - val_accuracy: 0.8151
Epoch 224/1000
275/275 [==============================] - 0s 142us/step - loss: 0.3996 - accuracy: 0.7600 - val_loss: 0.3581 - val_accuracy: 0.8151
Epoch 225/1000
275/275 [==============================] - 0s 138us/step - loss: 0.3993 - accuracy: 0.7600 - val_loss: 0.3577 - val_accuracy: 0.8151
Epoch 226/1000
275/275 [==============================] - 0s 139us/step - loss: 0.3991 - accuracy: 0.7600 - val_loss: 0.3572 - val_accuracy: 0.8151
Epoch 227/1000
275/275 [==============================] - 0s 141us/step - loss: 0.3989 - accuracy: 0.7600 - val_loss: 0.3570 - val_accuracy: 0.8151
Epoch 228/1000
275/275 [==============================] - 0s 135us/step - loss: 0.3987 - accuracy: 0.7600 - val_loss: 0.3566 - 

...

275/275 [==============================] - 0s 148us/step - loss: 0.2760 - accuracy: 0.8582 - val_loss: 0.2996 - val_accuracy: 0.8403
Epoch 773/1000
275/275 [==============================] - 0s 139us/step - loss: 0.2759 - accuracy: 0.8582 - val_loss: 0.2997 - val_accuracy: 0.8403
Epoch 774/1000
275/275 [==============================] - ETA: 0s - loss: 0.3258 - accuracy: 0.75 - 0s 139us/step - loss: 0.2751 - accuracy: 0.8582 - val_loss: 0.2995 - val_accuracy: 0.8403
Epoch 775/1000
275/275 [==============================] - 0s 143us/step - loss: 0.2748 - accuracy: 0.8582 - val_loss: 0.3000 - val_accuracy: 0.8403
Epoch 776/1000
275/275 [==============================] - 0s 133us/step - loss: 0.2746 - accuracy: 0.8582 - val_loss: 0.3001 - val_accuracy: 0.8403
Epoch 777/1000
275/275 [==============================] - 0s 141us/step - loss: 0.2745 - accuracy: 0.8582 - val_loss: 0.3005 - val_accuracy: 0.8403
Epoch 778/1000
275/275 [==============================] - 0s 138us/step - loss: 0.274

275/275 [==============================] - 0s 140us/step - loss: 0.2933 - accuracy: 0.8582 - val_loss: 0.3254 - val_accuracy: 0.8403
Epoch 828/1000
275/275 [==============================] - 0s 147us/step - loss: 0.2934 - accuracy: 0.8582 - val_loss: 0.3248 - val_accuracy: 0.8403
Epoch 829/1000
275/275 [==============================] - 0s 139us/step - loss: 0.2933 - accuracy: 0.8582 - val_loss: 0.3254 - val_accuracy: 0.8403
Epoch 830/1000
275/275 [==============================] - 0s 151us/step - loss: 0.2933 - accuracy: 0.8582 - val_loss: 0.3247 - val_accuracy: 0.8487
Epoch 831/1000
275/275 [==============================] - 0s 135us/step - loss: 0.2933 - accuracy: 0.8582 - val_loss: 0.3252 - val_accuracy: 0.8403
Epoch 832/1000
275/275 [==============================] - 0s 147us/step - loss: 0.2933 - accuracy: 0.8582 - val_loss: 0.3251 - val_accuracy: 0.8487
Epoch 833/1000
275/275 [==============================] - 0s 134us/step - loss: 0.2933 - accuracy: 0.8582 - val_loss: 0.3257 - 

275/275 [==============================] - 0s 139us/step - loss: 0.2787 - accuracy: 0.8582 - val_loss: 0.3217 - val_accuracy: 0.8403
Epoch 883/1000
275/275 [==============================] - 0s 146us/step - loss: 0.2788 - accuracy: 0.8582 - val_loss: 0.3213 - val_accuracy: 0.8403
Epoch 884/1000
275/275 [==============================] - 0s 143us/step - loss: 0.2787 - accuracy: 0.8582 - val_loss: 0.3215 - val_accuracy: 0.8403
Epoch 885/1000
275/275 [==============================] - 0s 130us/step - loss: 0.2786 - accuracy: 0.8582 - val_loss: 0.3212 - val_accuracy: 0.8403
Epoch 886/1000
275/275 [==============================] - 0s 136us/step - loss: 0.2786 - accuracy: 0.8582 - val_loss: 0.3207 - val_accuracy: 0.8403
Epoch 887/1000
275/275 [==============================] - 0s 141us/step - loss: 0.2787 - accuracy: 0.8582 - val_loss: 0.3211 - val_accuracy: 0.8403
Epoch 888/1000
275/275 [==============================] - 0s 134us/step - loss: 0.2787 - accuracy: 0.8582 - val_loss: 0.3199 - 

275/275 [==============================] - 0s 149us/step - loss: 0.2727 - accuracy: 0.8582 - val_loss: 0.3055 - val_accuracy: 0.8487
Epoch 938/1000
275/275 [==============================] - 0s 144us/step - loss: 0.2727 - accuracy: 0.8582 - val_loss: 0.3054 - val_accuracy: 0.8487
Epoch 939/1000
275/275 [==============================] - 0s 135us/step - loss: 0.2727 - accuracy: 0.8582 - val_loss: 0.3053 - val_accuracy: 0.8487
Epoch 940/1000
275/275 [==============================] - 0s 144us/step - loss: 0.2727 - accuracy: 0.8582 - val_loss: 0.3052 - val_accuracy: 0.8487
Epoch 941/1000
275/275 [==============================] - 0s 140us/step - loss: 0.2728 - accuracy: 0.8582 - val_loss: 0.3050 - val_accuracy: 0.8487
Epoch 942/1000
275/275 [==============================] - 0s 140us/step - loss: 0.2727 - accuracy: 0.8582 - val_loss: 0.3049 - val_accuracy: 0.8487
Epoch 943/1000
275/275 [==============================] - 0s 135us/step - loss: 0.2726 - accuracy: 0.8582 - val_loss: 0.3049 - 

275/275 [==============================] - 0s 135us/step - loss: 0.2714 - accuracy: 0.8582 - val_loss: 0.3019 - val_accuracy: 0.8487
Epoch 993/1000
275/275 [==============================] - 0s 142us/step - loss: 0.2713 - accuracy: 0.8582 - val_loss: 0.3016 - val_accuracy: 0.8487
Epoch 994/1000
275/275 [==============================] - 0s 145us/step - loss: 0.2713 - accuracy: 0.8582 - val_loss: 0.3016 - val_accuracy: 0.8487
Epoch 995/1000
275/275 [==============================] - 0s 134us/step - loss: 0.2713 - accuracy: 0.8582 - val_loss: 0.3015 - val_accuracy: 0.8487
Epoch 996/1000
275/275 [==============================] - 0s 138us/step - loss: 0.2714 - accuracy: 0.8582 - val_loss: 0.3016 - val_accuracy: 0.8487
Epoch 997/1000
275/275 [==============================] - 0s 143us/step - loss: 0.2714 - accuracy: 0.8582 - val_loss: 0.3014 - val_accuracy: 0.8487
Epoch 998/1000
275/275 [==============================] - 0s 139us/step - loss: 0.2713 - accuracy: 0.8582 - val_loss: 0.3014 - 

In [40]:
acc_test_comb3 = model1_comb3.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

119/119 [==============================] - 0s 92us/step
combination test accuracy: 84.87%


In [41]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [42]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [43]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 333us/step - loss: 9.7995 - accuracy: 0.4804 - val_loss: 0.7052 - val_accuracy: 0.5105
Epoch 2/1000
331/331 [==============================] - 0s 92us/step - loss: 0.6768 - accuracy: 0.5106 - val_loss: 0.7280 - val_accuracy: 0.5315
Epoch 3/1000
331/331 [==============================] - 0s 94us/step - loss: 0.6655 - accuracy: 0.5015 - val_loss: 0.7082 - val_accuracy: 0.5455
Epoch 4/1000
331/331 [==============================] - 0s 97us/step - loss: 0.6544 - accuracy: 0.5045 - val_loss: 0.6762 - val_accuracy: 0.5245
Epoch 5/1000
331/331 [==============================] - 0s 107us/step - loss: 0.6391 - accuracy: 0.5196 - val_loss: 0.6996 - val_accuracy: 0.5175
Epoch 6/1000
331/331 [==============================] - 0s 111us/step - loss: 0.6353 - accuracy: 0.5257 - val_loss: 0.7021 - val_accuracy: 0.5105
Epoch 7/1000
331/331 [==============================] - 0s 105us/step - loss: 0.6

331/331 [==============================] - 0s 121us/step - loss: 0.4423 - accuracy: 0.7795 - val_loss: 0.5373 - val_accuracy: 0.6923
Epoch 57/1000
331/331 [==============================] - 0s 119us/step - loss: 0.4427 - accuracy: 0.7795 - val_loss: 0.5200 - val_accuracy: 0.6923
Epoch 58/1000
331/331 [==============================] - 0s 102us/step - loss: 0.4333 - accuracy: 0.7825 - val_loss: 0.5267 - val_accuracy: 0.6713
Epoch 59/1000
331/331 [==============================] - 0s 86us/step - loss: 0.4466 - accuracy: 0.7674 - val_loss: 0.5301 - val_accuracy: 0.6923
Epoch 60/1000
331/331 [==============================] - 0s 89us/step - loss: 0.4309 - accuracy: 0.7855 - val_loss: 0.5321 - val_accuracy: 0.6923
Epoch 61/1000
331/331 [==============================] - 0s 87us/step - loss: 0.4286 - accuracy: 0.7764 - val_loss: 0.4946 - val_accuracy: 0.7133
Epoch 62/1000
331/331 [==============================] - 0s 94us/step - loss: 0.4366 - accuracy: 0.7674 - val_loss: 0.5255 - val_accura

Epoch 112/1000
331/331 [==============================] - 0s 102us/step - loss: 0.3942 - accuracy: 0.8127 - val_loss: 0.4900 - val_accuracy: 0.7762
Epoch 113/1000
331/331 [==============================] - 0s 91us/step - loss: 0.3987 - accuracy: 0.8006 - val_loss: 0.4530 - val_accuracy: 0.7692
Epoch 114/1000
331/331 [==============================] - 0s 95us/step - loss: 0.3918 - accuracy: 0.8036 - val_loss: 0.4627 - val_accuracy: 0.7902
Epoch 115/1000
331/331 [==============================] - 0s 102us/step - loss: 0.4125 - accuracy: 0.7976 - val_loss: 0.5015 - val_accuracy: 0.7762
Epoch 116/1000
331/331 [==============================] - 0s 103us/step - loss: 0.3912 - accuracy: 0.8097 - val_loss: 0.4759 - val_accuracy: 0.7832
Epoch 117/1000
331/331 [==============================] - 0s 101us/step - loss: 0.3948 - accuracy: 0.8127 - val_loss: 0.5147 - val_accuracy: 0.7762
Epoch 118/1000
331/331 [==============================] - 0s 109us/step - loss: 0.3805 - accuracy: 0.8338 - val_lo

331/331 [==============================] - 0s 72us/step - loss: 0.3599 - accuracy: 0.8187 - val_loss: 0.4001 - val_accuracy: 0.7832
Epoch 168/1000
331/331 [==============================] - 0s 97us/step - loss: 0.3514 - accuracy: 0.8097 - val_loss: 0.4004 - val_accuracy: 0.7832
Epoch 169/1000
331/331 [==============================] - 0s 86us/step - loss: 0.3511 - accuracy: 0.8127 - val_loss: 0.3800 - val_accuracy: 0.8112
Epoch 170/1000
331/331 [==============================] - 0s 90us/step - loss: 0.3586 - accuracy: 0.8127 - val_loss: 0.3920 - val_accuracy: 0.8042
Epoch 171/1000
331/331 [==============================] - 0s 87us/step - loss: 0.3944 - accuracy: 0.7976 - val_loss: 0.3892 - val_accuracy: 0.8112
Epoch 172/1000
331/331 [==============================] - 0s 79us/step - loss: 0.3563 - accuracy: 0.8097 - val_loss: 0.4272 - val_accuracy: 0.7413
Epoch 173/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3586 - accuracy: 0.8036 - val_loss: 0.4165 - val_acc

Epoch 223/1000
331/331 [==============================] - 0s 97us/step - loss: 0.3345 - accuracy: 0.8308 - val_loss: 0.3953 - val_accuracy: 0.8042
Epoch 224/1000
331/331 [==============================] - 0s 90us/step - loss: 0.3365 - accuracy: 0.8278 - val_loss: 0.4576 - val_accuracy: 0.7972
Epoch 225/1000
331/331 [==============================] - 0s 89us/step - loss: 0.3322 - accuracy: 0.8308 - val_loss: 0.4361 - val_accuracy: 0.8042
Epoch 226/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4063 - accuracy: 0.7946 - val_loss: 0.4062 - val_accuracy: 0.8042
Epoch 227/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3386 - accuracy: 0.8308 - val_loss: 0.3805 - val_accuracy: 0.8042
Epoch 228/1000
331/331 [==============================] - 0s 89us/step - loss: 0.3451 - accuracy: 0.8187 - val_loss: 0.3856 - val_accuracy: 0.7413
Epoch 229/1000
331/331 [==============================] - 0s 86us/step - loss: 0.3403 - accuracy: 0.8248 - val_loss: 0

...

Epoch 727/1000
331/331 [==============================] - 0s 86us/step - loss: 0.4041 - accuracy: 0.7613 - val_loss: 0.3958 - val_accuracy: 0.7622
Epoch 728/1000
331/331 [==============================] - 0s 83us/step - loss: 0.4040 - accuracy: 0.7613 - val_loss: 0.3957 - val_accuracy: 0.7622
Epoch 729/1000
331/331 [==============================] - 0s 85us/step - loss: 0.4040 - accuracy: 0.7613 - val_loss: 0.3956 - val_accuracy: 0.7622
Epoch 730/1000
331/331 [==============================] - 0s 89us/step - loss: 0.4040 - accuracy: 0.7613 - val_loss: 0.3954 - val_accuracy: 0.7622
Epoch 731/1000
331/331 [==============================] - 0s 84us/step - loss: 0.4039 - accuracy: 0.7613 - val_loss: 0.3953 - val_accuracy: 0.7622
Epoch 732/1000
331/331 [==============================] - 0s 86us/step - loss: 0.4038 - accuracy: 0.7613 - val_loss: 0.3952 - val_accuracy: 0.7622
Epoch 733/1000
331/331 [==============================] - 0s 78us/step - loss: 0.4038 - accuracy: 0.7613 - val_loss: 0

Epoch 783/1000
331/331 [==============================] - 0s 88us/step - loss: 0.3473 - accuracy: 0.8308 - val_loss: 0.3665 - val_accuracy: 0.7972
Epoch 784/1000
331/331 [==============================] - 0s 85us/step - loss: 0.3448 - accuracy: 0.8338 - val_loss: 0.3692 - val_accuracy: 0.7972
Epoch 785/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3464 - accuracy: 0.8369 - val_loss: 0.3650 - val_accuracy: 0.7832
Epoch 786/1000
331/331 [==============================] - 0s 85us/step - loss: 0.3473 - accuracy: 0.8308 - val_loss: 0.3655 - val_accuracy: 0.7832
Epoch 787/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3469 - accuracy: 0.8338 - val_loss: 0.3680 - val_accuracy: 0.7902
Epoch 788/1000
331/331 [==============================] - 0s 83us/step - loss: 0.3488 - accuracy: 0.8369 - val_loss: 0.3701 - val_accuracy: 0.7692
Epoch 789/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3447 - accuracy: 0.8369 - val_loss: 0

Epoch 839/1000
331/331 [==============================] - 0s 91us/step - loss: 0.3929 - accuracy: 0.7734 - val_loss: 0.3667 - val_accuracy: 0.8112
Epoch 840/1000
331/331 [==============================] - 0s 92us/step - loss: 0.3982 - accuracy: 0.7885 - val_loss: 0.3978 - val_accuracy: 0.7622
Epoch 841/1000
331/331 [==============================] - 0s 89us/step - loss: 0.4038 - accuracy: 0.7644 - val_loss: 0.3971 - val_accuracy: 0.7622
Epoch 842/1000
331/331 [==============================] - 0s 79us/step - loss: 0.4032 - accuracy: 0.7674 - val_loss: 0.3964 - val_accuracy: 0.7622
Epoch 843/1000
331/331 [==============================] - 0s 85us/step - loss: 0.4028 - accuracy: 0.7674 - val_loss: 0.3959 - val_accuracy: 0.7622
Epoch 844/1000
331/331 [==============================] - 0s 82us/step - loss: 0.4023 - accuracy: 0.7674 - val_loss: 0.3954 - val_accuracy: 0.7622
Epoch 845/1000
331/331 [==============================] - 0s 90us/step - loss: 0.4020 - accuracy: 0.7674 - val_loss: 0

Epoch 895/1000
331/331 [==============================] - 0s 86us/step - loss: 0.4020 - accuracy: 0.7644 - val_loss: 0.3945 - val_accuracy: 0.7622
Epoch 896/1000
331/331 [==============================] - 0s 94us/step - loss: 0.4000 - accuracy: 0.7644 - val_loss: 0.3937 - val_accuracy: 0.7622
Epoch 897/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3993 - accuracy: 0.7674 - val_loss: 0.3933 - val_accuracy: 0.7622
Epoch 898/1000
331/331 [==============================] - 0s 87us/step - loss: 0.3990 - accuracy: 0.7674 - val_loss: 0.3926 - val_accuracy: 0.7622
Epoch 899/1000
331/331 [==============================] - 0s 79us/step - loss: 0.3985 - accuracy: 0.7674 - val_loss: 0.3924 - val_accuracy: 0.7622
Epoch 900/1000
331/331 [==============================] - 0s 83us/step - loss: 0.3984 - accuracy: 0.7674 - val_loss: 0.3920 - val_accuracy: 0.7622
Epoch 901/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3981 - accuracy: 0.7674 - val_loss: 0

Epoch 951/1000
331/331 [==============================] - 0s 85us/step - loss: 0.3968 - accuracy: 0.7674 - val_loss: 0.3905 - val_accuracy: 0.7622
Epoch 952/1000
331/331 [==============================] - 0s 94us/step - loss: 0.3952 - accuracy: 0.7704 - val_loss: 0.3832 - val_accuracy: 0.7832
Epoch 953/1000
331/331 [==============================] - 0s 95us/step - loss: 0.3761 - accuracy: 0.8006 - val_loss: 0.3647 - val_accuracy: 0.8042
Epoch 954/1000
331/331 [==============================] - 0s 88us/step - loss: 0.3916 - accuracy: 0.7704 - val_loss: 0.3837 - val_accuracy: 0.7832
Epoch 955/1000
331/331 [==============================] - 0s 86us/step - loss: 0.4061 - accuracy: 0.7674 - val_loss: 0.3967 - val_accuracy: 0.7622
Epoch 956/1000
331/331 [==============================] - 0s 77us/step - loss: 0.4010 - accuracy: 0.7644 - val_loss: 0.3961 - val_accuracy: 0.7622
Epoch 957/1000
331/331 [==============================] - 0s 90us/step - loss: 0.4007 - accuracy: 0.7644 - val_loss: 0

In [44]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

143/143 [==============================] - 0s 74us/step
over-sampling test accuracy: 76.22%


In [69]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [70]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [71]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 395us/step - loss: 5.1434 - accuracy: 0.5076 - val_loss: 0.7859 - val_accuracy: 0.3986
Epoch 2/1000
331/331 [==============================] - 0s 64us/step - loss: 1.0619 - accuracy: 0.5076 - val_loss: 1.1119 - val_accuracy: 0.4685
Epoch 3/1000
331/331 [==============================] - 0s 87us/step - loss: 1.0527 - accuracy: 0.5166 - val_loss: 0.7968 - val_accuracy: 0.5734
Epoch 4/1000
331/331 [==============================] - 0s 84us/step - loss: 0.9069 - accuracy: 0.5227 - val_loss: 0.7125 - val_accuracy: 0.5944
Epoch 5/1000
331/331 [==============================] - 0s 84us/step - loss: 0.7364 - accuracy: 0.5650 - val_loss: 0.7014 - val_accuracy: 0.5874
Epoch 6/1000
331/331 [==============================] - 0s 79us/step - loss: 0.6663 - accuracy: 0.5378 - val_loss: 0.9387 - val_accuracy: 0.6014
Epoch 7/1000
331/331 [==============================] - 0s 91us/step - loss: 0.7553

Epoch 57/1000
331/331 [==============================] - 0s 85us/step - loss: 0.5510 - accuracy: 0.7644 - val_loss: 0.5882 - val_accuracy: 0.6923
Epoch 58/1000
331/331 [==============================] - 0s 92us/step - loss: 0.5313 - accuracy: 0.7492 - val_loss: 0.6223 - val_accuracy: 0.6993
Epoch 59/1000
331/331 [==============================] - 0s 75us/step - loss: 0.5320 - accuracy: 0.7704 - val_loss: 0.5997 - val_accuracy: 0.7063
Epoch 60/1000
331/331 [==============================] - 0s 78us/step - loss: 0.5320 - accuracy: 0.7492 - val_loss: 0.5711 - val_accuracy: 0.7203
Epoch 61/1000
331/331 [==============================] - 0s 80us/step - loss: 0.5346 - accuracy: 0.7221 - val_loss: 0.5693 - val_accuracy: 0.7203
Epoch 62/1000
331/331 [==============================] - 0s 101us/step - loss: 0.5215 - accuracy: 0.7341 - val_loss: 0.5834 - val_accuracy: 0.7413
Epoch 63/1000
331/331 [==============================] - 0s 72us/step - loss: 0.5223 - accuracy: 0.7613 - val_loss: 0.5926 

331/331 [==============================] - 0s 55us/step - loss: 0.4452 - accuracy: 0.7885 - val_loss: 0.5139 - val_accuracy: 0.7832
Epoch 114/1000
331/331 [==============================] - 0s 62us/step - loss: 0.4136 - accuracy: 0.8097 - val_loss: 0.4699 - val_accuracy: 0.7902
Epoch 115/1000
331/331 [==============================] - 0s 58us/step - loss: 0.4122 - accuracy: 0.8218 - val_loss: 0.4554 - val_accuracy: 0.7832
Epoch 116/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4432 - accuracy: 0.7795 - val_loss: 0.5199 - val_accuracy: 0.7972
Epoch 117/1000
331/331 [==============================] - 0s 61us/step - loss: 0.4119 - accuracy: 0.8338 - val_loss: 0.4698 - val_accuracy: 0.7692
Epoch 118/1000
331/331 [==============================] - 0s 60us/step - loss: 0.4243 - accuracy: 0.8006 - val_loss: 0.4816 - val_accuracy: 0.7832
Epoch 119/1000
331/331 [==============================] - 0s 60us/step - loss: 0.4106 - accuracy: 0.8066 - val_loss: 0.4618 - val_acc

Epoch 169/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3894 - accuracy: 0.8187 - val_loss: 0.5200 - val_accuracy: 0.7552
Epoch 170/1000
331/331 [==============================] - 0s 98us/step - loss: 0.4207 - accuracy: 0.7704 - val_loss: 0.4337 - val_accuracy: 0.8042
Epoch 171/1000
331/331 [==============================] - 0s 101us/step - loss: 0.3851 - accuracy: 0.8278 - val_loss: 0.4628 - val_accuracy: 0.7832
Epoch 172/1000
331/331 [==============================] - 0s 99us/step - loss: 0.3865 - accuracy: 0.8187 - val_loss: 0.4306 - val_accuracy: 0.8112
Epoch 173/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3776 - accuracy: 0.8218 - val_loss: 0.4417 - val_accuracy: 0.8112
Epoch 174/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3734 - accuracy: 0.8248 - val_loss: 0.4061 - val_accuracy: 0.7902
Epoch 175/1000
331/331 [==============================] - 0s 66us/step - loss: 0.3857 - accuracy: 0.8006 - val_loss: 

331/331 [==============================] - 0s 112us/step - loss: 0.3435 - accuracy: 0.8278 - val_loss: 0.3748 - val_accuracy: 0.8112
Epoch 225/1000
331/331 [==============================] - 0s 93us/step - loss: 0.4094 - accuracy: 0.7704 - val_loss: 0.4499 - val_accuracy: 0.7692
Epoch 226/1000
331/331 [==============================] - 0s 91us/step - loss: 0.3595 - accuracy: 0.8097 - val_loss: 0.4315 - val_accuracy: 0.8042
Epoch 227/1000
331/331 [==============================] - 0s 101us/step - loss: 0.3450 - accuracy: 0.8278 - val_loss: 0.4594 - val_accuracy: 0.7692
Epoch 228/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3559 - accuracy: 0.8248 - val_loss: 0.3999 - val_accuracy: 0.8112
Epoch 229/1000
331/331 [==============================] - 0s 97us/step - loss: 0.3416 - accuracy: 0.8308 - val_loss: 0.3926 - val_accuracy: 0.8112
Epoch 230/1000
331/331 [==============================] - 0s 102us/step - loss: 0.3420 - accuracy: 0.8308 - val_loss: 0.3831 - val_

...

Epoch 726/1000
331/331 [==============================] - 0s 65us/step - loss: 0.2851 - accuracy: 0.8489 - val_loss: 0.3363 - val_accuracy: 0.8112
Epoch 727/1000
331/331 [==============================] - 0s 78us/step - loss: 0.2876 - accuracy: 0.8459 - val_loss: 0.3149 - val_accuracy: 0.8112
Epoch 728/1000
331/331 [==============================] - 0s 67us/step - loss: 0.2888 - accuracy: 0.8459 - val_loss: 0.3370 - val_accuracy: 0.8112
Epoch 729/1000
331/331 [==============================] - 0s 78us/step - loss: 0.2886 - accuracy: 0.8459 - val_loss: 0.3520 - val_accuracy: 0.8112
Epoch 730/1000
331/331 [==============================] - 0s 66us/step - loss: 0.2859 - accuracy: 0.8459 - val_loss: 0.3170 - val_accuracy: 0.8112
Epoch 731/1000
331/331 [==============================] - 0s 74us/step - loss: 0.2833 - accuracy: 0.8520 - val_loss: 0.3358 - val_accuracy: 0.8112
Epoch 732/1000
331/331 [==============================] - 0s 64us/step - loss: 0.2935 - accuracy: 0.8429 - val_loss: 0

Epoch 782/1000
331/331 [==============================] - 0s 70us/step - loss: 0.2825 - accuracy: 0.8459 - val_loss: 0.3596 - val_accuracy: 0.8112
Epoch 783/1000
331/331 [==============================] - 0s 75us/step - loss: 0.2851 - accuracy: 0.8459 - val_loss: 0.3214 - val_accuracy: 0.8112
Epoch 784/1000
331/331 [==============================] - 0s 85us/step - loss: 0.2804 - accuracy: 0.8489 - val_loss: 0.3142 - val_accuracy: 0.8392
Epoch 785/1000
331/331 [==============================] - 0s 72us/step - loss: 0.2821 - accuracy: 0.8520 - val_loss: 0.3088 - val_accuracy: 0.8112
Epoch 786/1000
331/331 [==============================] - 0s 80us/step - loss: 0.2820 - accuracy: 0.8489 - val_loss: 0.4784 - val_accuracy: 0.8042
Epoch 787/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3037 - accuracy: 0.8399 - val_loss: 0.3339 - val_accuracy: 0.8112
Epoch 788/1000
331/331 [==============================] - 0s 85us/step - loss: 0.2867 - accuracy: 0.8489 - val_loss: 0

Epoch 838/1000
331/331 [==============================] - 0s 66us/step - loss: 0.3000 - accuracy: 0.8338 - val_loss: 0.3563 - val_accuracy: 0.8112
Epoch 839/1000
331/331 [==============================] - 0s 60us/step - loss: 0.2987 - accuracy: 0.8338 - val_loss: 0.3360 - val_accuracy: 0.8112
Epoch 840/1000
331/331 [==============================] - 0s 57us/step - loss: 0.2956 - accuracy: 0.8429 - val_loss: 0.3419 - val_accuracy: 0.8112
Epoch 841/1000
331/331 [==============================] - 0s 56us/step - loss: 0.2953 - accuracy: 0.8399 - val_loss: 0.3361 - val_accuracy: 0.8112
Epoch 842/1000
331/331 [==============================] - 0s 57us/step - loss: 0.2921 - accuracy: 0.8489 - val_loss: 0.3296 - val_accuracy: 0.7902
Epoch 843/1000
331/331 [==============================] - 0s 55us/step - loss: 0.2962 - accuracy: 0.8399 - val_loss: 0.3879 - val_accuracy: 0.7762
Epoch 844/1000
331/331 [==============================] - 0s 101us/step - loss: 0.3079 - accuracy: 0.8218 - val_loss: 

Epoch 894/1000
331/331 [==============================] - 0s 64us/step - loss: 0.2728 - accuracy: 0.8550 - val_loss: 0.3465 - val_accuracy: 0.8182
Epoch 895/1000
331/331 [==============================] - 0s 83us/step - loss: 0.2806 - accuracy: 0.8489 - val_loss: 0.3154 - val_accuracy: 0.8182
Epoch 896/1000
331/331 [==============================] - 0s 66us/step - loss: 0.2677 - accuracy: 0.8520 - val_loss: 0.3198 - val_accuracy: 0.8182
Epoch 897/1000
331/331 [==============================] - 0s 72us/step - loss: 0.2697 - accuracy: 0.8640 - val_loss: 0.3009 - val_accuracy: 0.8322
Epoch 898/1000
331/331 [==============================] - 0s 65us/step - loss: 0.2693 - accuracy: 0.8610 - val_loss: 0.2990 - val_accuracy: 0.8392
Epoch 899/1000
331/331 [==============================] - 0s 72us/step - loss: 0.2976 - accuracy: 0.8399 - val_loss: 0.3004 - val_accuracy: 0.7972
Epoch 900/1000
331/331 [==============================] - 0s 61us/step - loss: 0.2902 - accuracy: 0.8338 - val_loss: 0

Epoch 950/1000
331/331 [==============================] - 0s 71us/step - loss: 0.2932 - accuracy: 0.8399 - val_loss: 0.3455 - val_accuracy: 0.8112
Epoch 951/1000
331/331 [==============================] - 0s 71us/step - loss: 0.2936 - accuracy: 0.8338 - val_loss: 0.3354 - val_accuracy: 0.8112
Epoch 952/1000
331/331 [==============================] - 0s 74us/step - loss: 0.2987 - accuracy: 0.8338 - val_loss: 0.3280 - val_accuracy: 0.8112
Epoch 953/1000
331/331 [==============================] - 0s 71us/step - loss: 0.2909 - accuracy: 0.8399 - val_loss: 0.3374 - val_accuracy: 0.8112
Epoch 954/1000
331/331 [==============================] - 0s 77us/step - loss: 0.2964 - accuracy: 0.8369 - val_loss: 0.3459 - val_accuracy: 0.8112
Epoch 955/1000
331/331 [==============================] - 0s 65us/step - loss: 0.2947 - accuracy: 0.8369 - val_loss: 0.3382 - val_accuracy: 0.8112
Epoch 956/1000
331/331 [==============================] - 0s 70us/step - loss: 0.2921 - accuracy: 0.8399 - val_loss: 0

In [72]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 76us/step
over-sampling test accuracy: 81.12%


In [73]:
############ Feature selection using lasso ##########

In [74]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [75]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [76]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [77]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [78]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 539
selected features: 28


In [79]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  2,   5,  20,  42,  43, 115, 117, 166, 173, 202, 203, 205, 206,
        209, 215, 222, 240, 246, 268, 273, 301, 323, 329, 336, 346, 372,
        537, 538]])

In [80]:
names_arr = np.array(names)
names_arr[cols]

array(['TGGGTCTGAC', 'TAGTCGCACT', 'X1_14177_G_A', 'X1_118393_A_G',
       'X1_118483_T_C', 'X1_429389_T_C', 'X1_434861_C_T', 'X1_741250_T_C',
       'X1_775214_A_G', 'group_2982', 'group_1238', 'group_2369',
       'group_1488', 'group_11116', 'hlb', 'group_2110', 'group_4324',
       'group_226', 'group_1279', 'group_1097', 'argE', 'macB',
       'group_441', 'group_8377', 'group_4766', 'group_1659', 'ST', 'CC'],
      dtype='<U100')

In [81]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([  0,   1,   3,   4,   6,   7,   8,   9,  10,  11,
            ...
            527, 528, 529, 530, 531, 532, 533, 534, 535, 536],
           dtype='int64', length=511)

In [82]:
len(removed_features)

511

In [83]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 28) (255,) (255, 29)


In [84]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 192), (1, 180)]


In [85]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 237), (1, 237)]


In [86]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [87]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [88]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [89]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [90]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 260 samples, validate on 112 samples
Epoch 1/1000
260/260 [==============================] - 0s 388us/step - loss: 18.4016 - accuracy: 0.5538 - val_loss: 0.7044 - val_accuracy: 0.5804
Epoch 2/1000
260/260 [==============================] - 0s 79us/step - loss: 0.6755 - accuracy: 0.5462 - val_loss: 0.6921 - val_accuracy: 0.6250
Epoch 3/1000
260/260 [==============================] - 0s 114us/step - loss: 0.6462 - accuracy: 0.6154 - val_loss: 0.6600 - val_accuracy: 0.6964
Epoch 4/1000
260/260 [==============================] - 0s 126us/step - loss: 0.6770 - accuracy: 0.6462 - val_loss: 0.6705 - val_accuracy: 0.7232
Epoch 5/1000
260/260 [==============================] - 0s 118us/step - loss: 0.6509 - accuracy: 0.6462 - val_loss: 0.6483 - val_accuracy: 0.5357
Epoch 6/1000
260/260 [==============================] - 0s 112us/step - loss: 0.6310 - accuracy: 0.6192 - val_loss: 0.7121 - val_accuracy: 0.5536
Epoch 7/1000
260/260 [==============================] - 0s 107us/step - loss: 

260/260 [==============================] - 0s 110us/step - loss: 0.4684 - accuracy: 0.7769 - val_loss: 0.5494 - val_accuracy: 0.7679
Epoch 57/1000
260/260 [==============================] - 0s 131us/step - loss: 0.5281 - accuracy: 0.7654 - val_loss: 0.5105 - val_accuracy: 0.6786
Epoch 58/1000
260/260 [==============================] - 0s 98us/step - loss: 0.4540 - accuracy: 0.7769 - val_loss: 0.4881 - val_accuracy: 0.7589
Epoch 59/1000
260/260 [==============================] - 0s 86us/step - loss: 0.4527 - accuracy: 0.7846 - val_loss: 0.4904 - val_accuracy: 0.8036
Epoch 60/1000
260/260 [==============================] - 0s 93us/step - loss: 0.4745 - accuracy: 0.7923 - val_loss: 0.4844 - val_accuracy: 0.7589
Epoch 61/1000
260/260 [==============================] - 0s 87us/step - loss: 0.4559 - accuracy: 0.7885 - val_loss: 0.5902 - val_accuracy: 0.6429
Epoch 62/1000
260/260 [==============================] - 0s 90us/step - loss: 0.4784 - accuracy: 0.7692 - val_loss: 0.5634 - val_accurac

Epoch 112/1000
260/260 [==============================] - 0s 160us/step - loss: 0.4116 - accuracy: 0.8154 - val_loss: 0.5295 - val_accuracy: 0.7857
Epoch 113/1000
260/260 [==============================] - 0s 202us/step - loss: 0.4639 - accuracy: 0.7962 - val_loss: 0.4472 - val_accuracy: 0.8036
Epoch 114/1000
260/260 [==============================] - 0s 161us/step - loss: 0.4166 - accuracy: 0.8038 - val_loss: 0.4479 - val_accuracy: 0.8036
Epoch 115/1000
260/260 [==============================] - 0s 131us/step - loss: 0.4346 - accuracy: 0.7885 - val_loss: 0.4486 - val_accuracy: 0.8036
Epoch 116/1000
260/260 [==============================] - 0s 129us/step - loss: 0.4402 - accuracy: 0.7846 - val_loss: 0.4669 - val_accuracy: 0.7768
Epoch 117/1000
260/260 [==============================] - 0s 148us/step - loss: 0.4085 - accuracy: 0.8154 - val_loss: 0.5146 - val_accuracy: 0.7857
Epoch 118/1000
260/260 [==============================] - 0s 119us/step - loss: 0.4284 - accuracy: 0.7885 - val_

260/260 [==============================] - 0s 97us/step - loss: 0.4189 - accuracy: 0.8385 - val_loss: 0.4452 - val_accuracy: 0.7768
Epoch 168/1000
260/260 [==============================] - 0s 84us/step - loss: 0.4215 - accuracy: 0.8115 - val_loss: 0.5480 - val_accuracy: 0.7768
Epoch 169/1000
260/260 [==============================] - 0s 78us/step - loss: 0.4373 - accuracy: 0.8077 - val_loss: 0.4366 - val_accuracy: 0.7768
Epoch 170/1000
260/260 [==============================] - 0s 75us/step - loss: 0.3949 - accuracy: 0.8423 - val_loss: 0.4780 - val_accuracy: 0.7589
Epoch 171/1000
260/260 [==============================] - 0s 97us/step - loss: 0.4452 - accuracy: 0.8154 - val_loss: 0.4455 - val_accuracy: 0.7946
Epoch 172/1000
260/260 [==============================] - 0s 93us/step - loss: 0.3917 - accuracy: 0.8346 - val_loss: 0.4260 - val_accuracy: 0.8036
Epoch 173/1000
260/260 [==============================] - 0s 85us/step - loss: 0.3845 - accuracy: 0.8423 - val_loss: 0.5732 - val_acc

Epoch 223/1000
260/260 [==============================] - 0s 74us/step - loss: 0.7896 - accuracy: 0.7692 - val_loss: 0.4463 - val_accuracy: 0.7589
Epoch 224/1000
260/260 [==============================] - 0s 90us/step - loss: 0.4144 - accuracy: 0.8308 - val_loss: 0.4349 - val_accuracy: 0.7768
Epoch 225/1000
260/260 [==============================] - 0s 89us/step - loss: 0.3910 - accuracy: 0.8385 - val_loss: 0.4116 - val_accuracy: 0.8036
Epoch 226/1000
260/260 [==============================] - 0s 86us/step - loss: 0.3894 - accuracy: 0.8385 - val_loss: 0.4305 - val_accuracy: 0.7768
Epoch 227/1000
260/260 [==============================] - 0s 95us/step - loss: 0.3866 - accuracy: 0.8500 - val_loss: 0.4133 - val_accuracy: 0.8125
Epoch 228/1000
260/260 [==============================] - 0s 83us/step - loss: 0.3931 - accuracy: 0.8269 - val_loss: 0.4199 - val_accuracy: 0.7768
Epoch 229/1000
260/260 [==============================] - 0s 85us/step - loss: 0.3783 - accuracy: 0.8308 - val_loss: 0

...

Epoch 726/1000
260/260 [==============================] - 0s 119us/step - loss: 0.2551 - accuracy: 0.8615 - val_loss: 0.3059 - val_accuracy: 0.8125
Epoch 727/1000
260/260 [==============================] - 0s 121us/step - loss: 0.2580 - accuracy: 0.8577 - val_loss: 0.3002 - val_accuracy: 0.8304
Epoch 728/1000
260/260 [==============================] - 0s 155us/step - loss: 0.2534 - accuracy: 0.8615 - val_loss: 0.3130 - val_accuracy: 0.8125
Epoch 729/1000
260/260 [==============================] - 0s 169us/step - loss: 0.2554 - accuracy: 0.8577 - val_loss: 0.2996 - val_accuracy: 0.8125
Epoch 730/1000
260/260 [==============================] - 0s 141us/step - loss: 0.2514 - accuracy: 0.8615 - val_loss: 0.3090 - val_accuracy: 0.8125
Epoch 731/1000
260/260 [==============================] - 0s 170us/step - loss: 0.3085 - accuracy: 0.8462 - val_loss: 0.3036 - val_accuracy: 0.8125
Epoch 732/1000
260/260 [==============================] - 0s 111us/step - loss: 0.2770 - accuracy: 0.8500 - val_

260/260 [==============================] - 0s 88us/step - loss: 0.3402 - accuracy: 0.8423 - val_loss: 0.3148 - val_accuracy: 0.8304
Epoch 782/1000
260/260 [==============================] - 0s 96us/step - loss: 0.2970 - accuracy: 0.8577 - val_loss: 0.3112 - val_accuracy: 0.8304
Epoch 783/1000
260/260 [==============================] - 0s 102us/step - loss: 0.2922 - accuracy: 0.8615 - val_loss: 0.3133 - val_accuracy: 0.8304
Epoch 784/1000
260/260 [==============================] - 0s 98us/step - loss: 0.2930 - accuracy: 0.8615 - val_loss: 0.3162 - val_accuracy: 0.8304
Epoch 785/1000
260/260 [==============================] - 0s 97us/step - loss: 0.2906 - accuracy: 0.8615 - val_loss: 0.3099 - val_accuracy: 0.8304
Epoch 786/1000
260/260 [==============================] - 0s 133us/step - loss: 0.2908 - accuracy: 0.8615 - val_loss: 0.3113 - val_accuracy: 0.8304
Epoch 787/1000
260/260 [==============================] - 0s 122us/step - loss: 0.2884 - accuracy: 0.8615 - val_loss: 0.3113 - val_

Epoch 837/1000
260/260 [==============================] - 0s 91us/step - loss: 0.3144 - accuracy: 0.8538 - val_loss: 0.2945 - val_accuracy: 0.8214
Epoch 838/1000
260/260 [==============================] - 0s 84us/step - loss: 0.2466 - accuracy: 0.8615 - val_loss: 0.3061 - val_accuracy: 0.8125
Epoch 839/1000
260/260 [==============================] - 0s 100us/step - loss: 0.2765 - accuracy: 0.8423 - val_loss: 0.3367 - val_accuracy: 0.8036
Epoch 840/1000
260/260 [==============================] - 0s 83us/step - loss: 0.3869 - accuracy: 0.8308 - val_loss: 0.2941 - val_accuracy: 0.8304
Epoch 841/1000
260/260 [==============================] - 0s 93us/step - loss: 0.2480 - accuracy: 0.8615 - val_loss: 0.2916 - val_accuracy: 0.8304
Epoch 842/1000
260/260 [==============================] - 0s 87us/step - loss: 0.2455 - accuracy: 0.8615 - val_loss: 0.2966 - val_accuracy: 0.8125
Epoch 843/1000
260/260 [==============================] - 0s 81us/step - loss: 0.2450 - accuracy: 0.8615 - val_loss: 

260/260 [==============================] - 0s 119us/step - loss: 0.2475 - accuracy: 0.8615 - val_loss: 0.6727 - val_accuracy: 0.7857
Epoch 893/1000
260/260 [==============================] - 0s 166us/step - loss: 0.2688 - accuracy: 0.8462 - val_loss: 0.2858 - val_accuracy: 0.8304
Epoch 894/1000
260/260 [==============================] - 0s 155us/step - loss: 0.2403 - accuracy: 0.8615 - val_loss: 0.3547 - val_accuracy: 0.8036
Epoch 895/1000
260/260 [==============================] - 0s 144us/step - loss: 0.2556 - accuracy: 0.8577 - val_loss: 0.2887 - val_accuracy: 0.8214
Epoch 896/1000
260/260 [==============================] - 0s 136us/step - loss: 0.2405 - accuracy: 0.8615 - val_loss: 0.2848 - val_accuracy: 0.8304
Epoch 897/1000
260/260 [==============================] - 0s 123us/step - loss: 0.2393 - accuracy: 0.8615 - val_loss: 0.4644 - val_accuracy: 0.7946
Epoch 898/1000
260/260 [==============================] - 0s 99us/step - loss: 0.2968 - accuracy: 0.8462 - val_loss: 0.3030 - v

Epoch 948/1000
260/260 [==============================] - 0s 81us/step - loss: 0.2409 - accuracy: 0.8615 - val_loss: 0.2788 - val_accuracy: 0.8214
Epoch 949/1000
260/260 [==============================] - 0s 87us/step - loss: 0.2379 - accuracy: 0.8615 - val_loss: 0.2824 - val_accuracy: 0.8304
Epoch 950/1000
260/260 [==============================] - 0s 90us/step - loss: 0.2381 - accuracy: 0.8615 - val_loss: 0.2875 - val_accuracy: 0.8125
Epoch 951/1000
260/260 [==============================] - 0s 85us/step - loss: 0.2379 - accuracy: 0.8615 - val_loss: 0.2799 - val_accuracy: 0.8304
Epoch 952/1000
260/260 [==============================] - 0s 90us/step - loss: 0.2364 - accuracy: 0.8615 - val_loss: 0.2834 - val_accuracy: 0.8214
Epoch 953/1000
260/260 [==============================] - 0s 85us/step - loss: 0.2395 - accuracy: 0.8615 - val_loss: 0.2859 - val_accuracy: 0.8125
Epoch 954/1000
260/260 [==============================] - 0s 91us/step - loss: 0.2387 - accuracy: 0.8615 - val_loss: 0

In [91]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


112/112 [==============================] - 0s 93us/step
combination test accuracy: 78.57%


In [104]:
#### improve neural network on combination data
model2_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [105]:
model2_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [106]:
model2_comb2.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 260 samples, validate on 112 samples
Epoch 1/1000
260/260 [==============================] - 0s 677us/step - loss: 10.8048 - accuracy: 0.4615 - val_loss: 2.5230 - val_accuracy: 0.5446
Epoch 2/1000
260/260 [==============================] - 0s 97us/step - loss: 2.7197 - accuracy: 0.5577 - val_loss: 0.8172 - val_accuracy: 0.4732
Epoch 3/1000
260/260 [==============================] - 0s 126us/step - loss: 2.5227 - accuracy: 0.5115 - val_loss: 0.7629 - val_accuracy: 0.5000
Epoch 4/1000
260/260 [==============================] - 0s 118us/step - loss: 2.1222 - accuracy: 0.5962 - val_loss: 0.7495 - val_accuracy: 0.5000
Epoch 5/1000
260/260 [==============================] - 0s 114us/step - loss: 1.9975 - accuracy: 0.6462 - val_loss: 0.7418 - val_accuracy: 0.5089
Epoch 6/1000
260/260 [==============================] - 0s 124us/step - loss: 1.8676 - accuracy: 0.6500 - val_loss: 0.7263 - val_accuracy: 0.5089
Epoch 7/1000
260/260 [==============================] - 0s 118us/step - loss: 

260/260 [==============================] - 0s 132us/step - loss: 1.9296 - accuracy: 0.7115 - val_loss: 0.6010 - val_accuracy: 0.5982
Epoch 57/1000
260/260 [==============================] - 0s 153us/step - loss: 2.2301 - accuracy: 0.6846 - val_loss: 0.5991 - val_accuracy: 0.6071
Epoch 58/1000
260/260 [==============================] - 0s 146us/step - loss: 2.0377 - accuracy: 0.7115 - val_loss: 0.7378 - val_accuracy: 0.6071
Epoch 59/1000
260/260 [==============================] - 0s 145us/step - loss: 1.6793 - accuracy: 0.7038 - val_loss: 0.5953 - val_accuracy: 0.6161
Epoch 60/1000
260/260 [==============================] - 0s 135us/step - loss: 2.3192 - accuracy: 0.6962 - val_loss: 0.5931 - val_accuracy: 0.6071
Epoch 61/1000
260/260 [==============================] - 0s 138us/step - loss: 1.9802 - accuracy: 0.7308 - val_loss: 0.5942 - val_accuracy: 0.5982
Epoch 62/1000
260/260 [==============================] - 0s 146us/step - loss: 1.5728 - accuracy: 0.7577 - val_loss: 0.5962 - val_ac

Epoch 112/1000
260/260 [==============================] - 0s 123us/step - loss: 1.7588 - accuracy: 0.7769 - val_loss: 0.5652 - val_accuracy: 0.7589
Epoch 113/1000
260/260 [==============================] - 0s 107us/step - loss: 1.6301 - accuracy: 0.7462 - val_loss: 0.5678 - val_accuracy: 0.7321
Epoch 114/1000
260/260 [==============================] - 0s 94us/step - loss: 2.1402 - accuracy: 0.7231 - val_loss: 0.5686 - val_accuracy: 0.6339
Epoch 115/1000
260/260 [==============================] - 0s 88us/step - loss: 2.0622 - accuracy: 0.7538 - val_loss: 0.6012 - val_accuracy: 0.6339
Epoch 116/1000
260/260 [==============================] - 0s 99us/step - loss: 1.9213 - accuracy: 0.7423 - val_loss: 0.5884 - val_accuracy: 0.6607
Epoch 117/1000
260/260 [==============================] - 0s 107us/step - loss: 1.7830 - accuracy: 0.7538 - val_loss: 0.5553 - val_accuracy: 0.6429
Epoch 118/1000
260/260 [==============================] - 0s 102us/step - loss: 2.0133 - accuracy: 0.7615 - val_los

260/260 [==============================] - 0s 86us/step - loss: 1.7391 - accuracy: 0.7500 - val_loss: 0.5068 - val_accuracy: 0.7589
Epoch 168/1000
260/260 [==============================] - 0s 94us/step - loss: 1.4551 - accuracy: 0.7615 - val_loss: 0.5052 - val_accuracy: 0.7679
Epoch 169/1000
260/260 [==============================] - 0s 88us/step - loss: 2.1606 - accuracy: 0.7077 - val_loss: 0.5046 - val_accuracy: 0.7679
Epoch 170/1000
260/260 [==============================] - 0s 102us/step - loss: 1.5764 - accuracy: 0.7500 - val_loss: 0.5040 - val_accuracy: 0.7679
Epoch 171/1000
260/260 [==============================] - 0s 85us/step - loss: 1.6058 - accuracy: 0.7192 - val_loss: 0.5022 - val_accuracy: 0.7589
Epoch 172/1000
260/260 [==============================] - 0s 91us/step - loss: 1.8541 - accuracy: 0.7385 - val_loss: 0.5009 - val_accuracy: 0.7589
Epoch 173/1000
260/260 [==============================] - 0s 97us/step - loss: 1.8172 - accuracy: 0.7269 - val_loss: 0.5054 - val_ac

Epoch 223/1000
260/260 [==============================] - 0s 101us/step - loss: 1.6142 - accuracy: 0.7692 - val_loss: 0.5138 - val_accuracy: 0.7857
Epoch 224/1000
260/260 [==============================] - 0s 107us/step - loss: 1.5542 - accuracy: 0.7654 - val_loss: 0.4702 - val_accuracy: 0.8036
Epoch 225/1000
260/260 [==============================] - 0s 99us/step - loss: 1.7248 - accuracy: 0.7500 - val_loss: 1.8014 - val_accuracy: 0.7054
Epoch 226/1000
260/260 [==============================] - 0s 96us/step - loss: 1.7982 - accuracy: 0.7192 - val_loss: 0.4755 - val_accuracy: 0.7679
Epoch 227/1000
260/260 [==============================] - 0s 98us/step - loss: 1.8475 - accuracy: 0.7269 - val_loss: 0.4758 - val_accuracy: 0.7679
Epoch 228/1000
260/260 [==============================] - 0s 89us/step - loss: 1.6032 - accuracy: 0.7538 - val_loss: 0.4750 - val_accuracy: 0.7679
Epoch 229/1000
260/260 [==============================] - 0s 96us/step - loss: 2.1450 - accuracy: 0.7000 - val_loss:

...

Epoch 724/1000
260/260 [==============================] - 0s 96us/step - loss: 1.6459 - accuracy: 0.7423 - val_loss: 0.4407 - val_accuracy: 0.7589
Epoch 725/1000
260/260 [==============================] - 0s 97us/step - loss: 2.0037 - accuracy: 0.7115 - val_loss: 0.4408 - val_accuracy: 0.7589
Epoch 726/1000
260/260 [==============================] - 0s 94us/step - loss: 1.7564 - accuracy: 0.7462 - val_loss: 0.4387 - val_accuracy: 0.7589
Epoch 727/1000
260/260 [==============================] - 0s 99us/step - loss: 1.8237 - accuracy: 0.7231 - val_loss: 0.4389 - val_accuracy: 0.7589
Epoch 728/1000
260/260 [==============================] - 0s 87us/step - loss: 2.1282 - accuracy: 0.6923 - val_loss: 0.4404 - val_accuracy: 0.7589
Epoch 729/1000
260/260 [==============================] - 0s 97us/step - loss: 1.4182 - accuracy: 0.7538 - val_loss: 0.4406 - val_accuracy: 0.7589
Epoch 730/1000
260/260 [==============================] - 0s 90us/step - loss: 1.5306 - accuracy: 0.7500 - val_loss: 0

260/260 [==============================] - 0s 97us/step - loss: 2.1021 - accuracy: 0.7154 - val_loss: 0.4404 - val_accuracy: 0.6875
Epoch 780/1000
260/260 [==============================] - 0s 105us/step - loss: 1.6295 - accuracy: 0.7538 - val_loss: 0.4399 - val_accuracy: 0.7589
Epoch 781/1000
260/260 [==============================] - 0s 102us/step - loss: 1.8814 - accuracy: 0.7269 - val_loss: 0.4394 - val_accuracy: 0.7589
Epoch 782/1000
260/260 [==============================] - 0s 107us/step - loss: 1.6770 - accuracy: 0.7615 - val_loss: 0.4400 - val_accuracy: 0.7589
Epoch 783/1000
260/260 [==============================] - 0s 87us/step - loss: 1.6563 - accuracy: 0.7308 - val_loss: 0.4458 - val_accuracy: 0.7589
Epoch 784/1000
260/260 [==============================] - 0s 96us/step - loss: 1.8854 - accuracy: 0.7192 - val_loss: 0.4422 - val_accuracy: 0.7589
Epoch 785/1000
260/260 [==============================] - 0s 104us/step - loss: 1.8370 - accuracy: 0.7115 - val_loss: 0.4417 - val

Epoch 835/1000
260/260 [==============================] - 0s 101us/step - loss: 2.2271 - accuracy: 0.6962 - val_loss: 0.4407 - val_accuracy: 0.6875
Epoch 836/1000
260/260 [==============================] - 0s 108us/step - loss: 1.6736 - accuracy: 0.7654 - val_loss: 0.4393 - val_accuracy: 0.6875
Epoch 837/1000
260/260 [==============================] - 0s 101us/step - loss: 1.8760 - accuracy: 0.7231 - val_loss: 0.4391 - val_accuracy: 0.6875
Epoch 838/1000
260/260 [==============================] - 0s 95us/step - loss: 1.2945 - accuracy: 0.7692 - val_loss: 0.4374 - val_accuracy: 0.7589
Epoch 839/1000
260/260 [==============================] - 0s 99us/step - loss: 1.5557 - accuracy: 0.7269 - val_loss: 0.4390 - val_accuracy: 0.7589
Epoch 840/1000
260/260 [==============================] - 0s 90us/step - loss: 1.9941 - accuracy: 0.7154 - val_loss: 0.4383 - val_accuracy: 0.7589
Epoch 841/1000
260/260 [==============================] - 0s 91us/step - loss: 1.8106 - accuracy: 0.7346 - val_loss

Epoch 891/1000
260/260 [==============================] - 0s 94us/step - loss: 2.1075 - accuracy: 0.7038 - val_loss: 0.4362 - val_accuracy: 0.7589
Epoch 892/1000
260/260 [==============================] - 0s 109us/step - loss: 2.0701 - accuracy: 0.6962 - val_loss: 0.4363 - val_accuracy: 0.7589
Epoch 893/1000
260/260 [==============================] - 0s 101us/step - loss: 1.9412 - accuracy: 0.7077 - val_loss: 0.4373 - val_accuracy: 0.7589
Epoch 894/1000
260/260 [==============================] - 0s 90us/step - loss: 1.4141 - accuracy: 0.7462 - val_loss: 0.4373 - val_accuracy: 0.7589
Epoch 895/1000
260/260 [==============================] - 0s 99us/step - loss: 2.1580 - accuracy: 0.7038 - val_loss: 0.4380 - val_accuracy: 0.6875
Epoch 896/1000
260/260 [==============================] - 0s 91us/step - loss: 1.4677 - accuracy: 0.7538 - val_loss: 0.4364 - val_accuracy: 0.7589
Epoch 897/1000
260/260 [==============================] - 0s 95us/step - loss: 2.2207 - accuracy: 0.7038 - val_loss:

Epoch 947/1000
260/260 [==============================] - 0s 103us/step - loss: 1.5448 - accuracy: 0.7269 - val_loss: 0.4363 - val_accuracy: 0.7589
Epoch 948/1000
260/260 [==============================] - 0s 101us/step - loss: 2.2577 - accuracy: 0.7192 - val_loss: 0.4363 - val_accuracy: 0.7589
Epoch 949/1000
260/260 [==============================] - 0s 90us/step - loss: 1.9302 - accuracy: 0.7231 - val_loss: 0.4377 - val_accuracy: 0.6875
Epoch 950/1000
260/260 [==============================] - 0s 100us/step - loss: 2.1266 - accuracy: 0.6923 - val_loss: 0.4378 - val_accuracy: 0.6875
Epoch 951/1000
260/260 [==============================] - 0s 102us/step - loss: 1.8832 - accuracy: 0.7192 - val_loss: 0.4367 - val_accuracy: 0.7589
Epoch 952/1000
260/260 [==============================] - 0s 91us/step - loss: 1.9917 - accuracy: 0.7154 - val_loss: 0.4355 - val_accuracy: 0.7589
Epoch 953/1000
260/260 [==============================] - 0s 97us/step - loss: 1.6900 - accuracy: 0.7462 - val_los

In [107]:
acc_test_comb3 = model2_comb2.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

112/112 [==============================] - 0s 92us/step
combination test accuracy: 76.79%


In [108]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [109]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [110]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 333us/step - loss: 8.1036 - accuracy: 0.5166 - val_loss: 0.7704 - val_accuracy: 0.5315
Epoch 2/1000
331/331 [==============================] - 0s 76us/step - loss: 0.7165 - accuracy: 0.4864 - val_loss: 0.8837 - val_accuracy: 0.3636
Epoch 3/1000
331/331 [==============================] - 0s 98us/step - loss: 0.7009 - accuracy: 0.4471 - val_loss: 0.6966 - val_accuracy: 0.5175
Epoch 4/1000
331/331 [==============================] - 0s 105us/step - loss: 0.6752 - accuracy: 0.5166 - val_loss: 0.7460 - val_accuracy: 0.5175
Epoch 5/1000
331/331 [==============================] - 0s 92us/step - loss: 0.6867 - accuracy: 0.4834 - val_loss: 0.7262 - val_accuracy: 0.5035
Epoch 6/1000
331/331 [==============================] - 0s 86us/step - loss: 0.6655 - accuracy: 0.5287 - val_loss: 0.7257 - val_accuracy: 0.5035
Epoch 7/1000
331/331 [==============================] - 0s 93us/step - loss: 0.676

331/331 [==============================] - 0s 110us/step - loss: 0.6024 - accuracy: 0.6677 - val_loss: 0.6440 - val_accuracy: 0.6434
Epoch 57/1000
331/331 [==============================] - 0s 103us/step - loss: 0.5927 - accuracy: 0.6556 - val_loss: 0.6492 - val_accuracy: 0.6364
Epoch 58/1000
331/331 [==============================] - 0s 86us/step - loss: 0.5922 - accuracy: 0.6798 - val_loss: 0.8511 - val_accuracy: 0.6084
Epoch 59/1000
331/331 [==============================] - 0s 90us/step - loss: 0.6187 - accuracy: 0.6798 - val_loss: 0.6535 - val_accuracy: 0.6364
Epoch 60/1000
331/331 [==============================] - 0s 84us/step - loss: 0.5845 - accuracy: 0.7069 - val_loss: 0.6836 - val_accuracy: 0.6084
Epoch 61/1000
331/331 [==============================] - 0s 86us/step - loss: 0.5802 - accuracy: 0.6949 - val_loss: 0.6984 - val_accuracy: 0.6084
Epoch 62/1000
331/331 [==============================] - 0s 81us/step - loss: 0.5799 - accuracy: 0.6949 - val_loss: 0.6473 - val_accurac

Epoch 112/1000
331/331 [==============================] - 0s 84us/step - loss: 0.5365 - accuracy: 0.7341 - val_loss: 0.5816 - val_accuracy: 0.6993
Epoch 113/1000
331/331 [==============================] - 0s 71us/step - loss: 0.5275 - accuracy: 0.7462 - val_loss: 0.5750 - val_accuracy: 0.6923
Epoch 114/1000
331/331 [==============================] - 0s 79us/step - loss: 0.5408 - accuracy: 0.7402 - val_loss: 0.6432 - val_accuracy: 0.6643
Epoch 115/1000
331/331 [==============================] - 0s 81us/step - loss: 0.5242 - accuracy: 0.7583 - val_loss: 0.5877 - val_accuracy: 0.6853
Epoch 116/1000
331/331 [==============================] - 0s 74us/step - loss: 0.5329 - accuracy: 0.7523 - val_loss: 1.0427 - val_accuracy: 0.6573
Epoch 117/1000
331/331 [==============================] - 0s 84us/step - loss: 0.5941 - accuracy: 0.7432 - val_loss: 0.6525 - val_accuracy: 0.6853
Epoch 118/1000
331/331 [==============================] - 0s 86us/step - loss: 0.5319 - accuracy: 0.7674 - val_loss: 0

Epoch 168/1000
331/331 [==============================] - 0s 78us/step - loss: 0.5133 - accuracy: 0.7553 - val_loss: 0.5976 - val_accuracy: 0.6993
Epoch 169/1000
331/331 [==============================] - 0s 87us/step - loss: 0.5329 - accuracy: 0.7372 - val_loss: 0.6059 - val_accuracy: 0.6993
Epoch 170/1000
331/331 [==============================] - 0s 77us/step - loss: 0.4923 - accuracy: 0.7764 - val_loss: 0.7137 - val_accuracy: 0.6643
Epoch 171/1000
331/331 [==============================] - 0s 76us/step - loss: 0.5403 - accuracy: 0.7553 - val_loss: 0.5745 - val_accuracy: 0.6643
Epoch 172/1000
331/331 [==============================] - 0s 80us/step - loss: 0.4951 - accuracy: 0.7734 - val_loss: 0.5716 - val_accuracy: 0.7063
Epoch 173/1000
331/331 [==============================] - 0s 82us/step - loss: 0.4921 - accuracy: 0.7583 - val_loss: 0.6300 - val_accuracy: 0.6923
Epoch 174/1000
331/331 [==============================] - 0s 75us/step - loss: 0.4851 - accuracy: 0.7795 - val_loss: 0

Epoch 224/1000
331/331 [==============================] - 0s 76us/step - loss: 0.4546 - accuracy: 0.7613 - val_loss: 0.5098 - val_accuracy: 0.7203
Epoch 225/1000
331/331 [==============================] - 0s 80us/step - loss: 0.4731 - accuracy: 0.7734 - val_loss: 0.5696 - val_accuracy: 0.7063
Epoch 226/1000
331/331 [==============================] - 0s 76us/step - loss: 0.4532 - accuracy: 0.7885 - val_loss: 0.5569 - val_accuracy: 0.7063
Epoch 227/1000
331/331 [==============================] - 0s 80us/step - loss: 0.4639 - accuracy: 0.7915 - val_loss: 0.6736 - val_accuracy: 0.6503
Epoch 228/1000
331/331 [==============================] - 0s 72us/step - loss: 0.4936 - accuracy: 0.7462 - val_loss: 0.5420 - val_accuracy: 0.6573
Epoch 229/1000
331/331 [==============================] - 0s 77us/step - loss: 0.5195 - accuracy: 0.7613 - val_loss: 0.5446 - val_accuracy: 0.7063
Epoch 230/1000
331/331 [==============================] - 0s 77us/step - loss: 0.5128 - accuracy: 0.7583 - val_loss: 0

...

Epoch 728/1000
331/331 [==============================] - 0s 78us/step - loss: 0.4051 - accuracy: 0.7946 - val_loss: 0.4341 - val_accuracy: 0.7832
Epoch 729/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3833 - accuracy: 0.8127 - val_loss: 0.5082 - val_accuracy: 0.7832
Epoch 730/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3830 - accuracy: 0.8187 - val_loss: 0.4666 - val_accuracy: 0.7273
Epoch 731/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3849 - accuracy: 0.8157 - val_loss: 0.4075 - val_accuracy: 0.7902
Epoch 732/1000
331/331 [==============================] - 0s 71us/step - loss: 0.4755 - accuracy: 0.8066 - val_loss: 0.4681 - val_accuracy: 0.7832
Epoch 733/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3987 - accuracy: 0.7915 - val_loss: 0.6944 - val_accuracy: 0.7483
Epoch 734/1000
331/331 [==============================] - 0s 76us/step - loss: 0.4019 - accuracy: 0.8006 - val_loss: 0

Epoch 784/1000
331/331 [==============================] - 0s 79us/step - loss: 0.3804 - accuracy: 0.8218 - val_loss: 0.4333 - val_accuracy: 0.7832
Epoch 785/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3827 - accuracy: 0.8097 - val_loss: 0.4234 - val_accuracy: 0.7832
Epoch 786/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3629 - accuracy: 0.8248 - val_loss: 0.4771 - val_accuracy: 0.7832
Epoch 787/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3780 - accuracy: 0.8157 - val_loss: 0.4269 - val_accuracy: 0.7832
Epoch 788/1000
331/331 [==============================] - 0s 78us/step - loss: 0.3854 - accuracy: 0.8127 - val_loss: 0.4052 - val_accuracy: 0.7832
Epoch 789/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3708 - accuracy: 0.8187 - val_loss: 0.6171 - val_accuracy: 0.7832
Epoch 790/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3875 - accuracy: 0.8127 - val_loss: 0

Epoch 840/1000
331/331 [==============================] - 0s 80us/step - loss: 0.3903 - accuracy: 0.8248 - val_loss: 0.4343 - val_accuracy: 0.7902
Epoch 841/1000
331/331 [==============================] - 0s 96us/step - loss: 0.3921 - accuracy: 0.8127 - val_loss: 0.4430 - val_accuracy: 0.7832
Epoch 842/1000
331/331 [==============================] - 0s 83us/step - loss: 0.3821 - accuracy: 0.8097 - val_loss: 0.4615 - val_accuracy: 0.7832
Epoch 843/1000
331/331 [==============================] - 0s 92us/step - loss: 0.3674 - accuracy: 0.8127 - val_loss: 0.4436 - val_accuracy: 0.7832
Epoch 844/1000
331/331 [==============================] - 0s 96us/step - loss: 0.3768 - accuracy: 0.8097 - val_loss: 0.4597 - val_accuracy: 0.7832
Epoch 845/1000
331/331 [==============================] - 0s 83us/step - loss: 0.3705 - accuracy: 0.8127 - val_loss: 0.4865 - val_accuracy: 0.7832
Epoch 846/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3704 - accuracy: 0.8097 - val_loss: 0

Epoch 896/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3581 - accuracy: 0.8218 - val_loss: 0.4018 - val_accuracy: 0.7972
Epoch 897/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3813 - accuracy: 0.8218 - val_loss: 0.4807 - val_accuracy: 0.7832
Epoch 898/1000
331/331 [==============================] - 0s 86us/step - loss: 0.3634 - accuracy: 0.8218 - val_loss: 2.5876 - val_accuracy: 0.7552
Epoch 899/1000
331/331 [==============================] - 0s 81us/step - loss: 0.8438 - accuracy: 0.7855 - val_loss: 0.8956 - val_accuracy: 0.6993
Epoch 900/1000
331/331 [==============================] - 0s 78us/step - loss: 0.4588 - accuracy: 0.7885 - val_loss: 0.4334 - val_accuracy: 0.7483
Epoch 901/1000
331/331 [==============================] - 0s 76us/step - loss: 0.3968 - accuracy: 0.7946 - val_loss: 0.6998 - val_accuracy: 0.7273
Epoch 902/1000
331/331 [==============================] - 0s 78us/step - loss: 0.4437 - accuracy: 0.8036 - val_loss: 0

Epoch 952/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3717 - accuracy: 0.8157 - val_loss: 0.5084 - val_accuracy: 0.7832
Epoch 953/1000
331/331 [==============================] - 0s 78us/step - loss: 0.3587 - accuracy: 0.8127 - val_loss: 0.4609 - val_accuracy: 0.7832
Epoch 954/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3490 - accuracy: 0.8278 - val_loss: 0.4370 - val_accuracy: 0.7832
Epoch 955/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3563 - accuracy: 0.8218 - val_loss: 0.4282 - val_accuracy: 0.7832
Epoch 956/1000
331/331 [==============================] - 0s 78us/step - loss: 0.3464 - accuracy: 0.8308 - val_loss: 0.4085 - val_accuracy: 0.7832
Epoch 957/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3521 - accuracy: 0.8218 - val_loss: 0.4684 - val_accuracy: 0.7832
Epoch 958/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3547 - accuracy: 0.8187 - val_loss: 0

In [111]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 74us/step
combination test accuracy: 78.32%


In [128]:
#### improve neural network on over-sampling data
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [129]:
model2_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [130]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 355us/step - loss: 10.1692 - accuracy: 0.5529 - val_loss: 0.6793 - val_accuracy: 0.5804
Epoch 2/1000
331/331 [==============================] - 0s 76us/step - loss: 0.6431 - accuracy: 0.6103 - val_loss: 0.6802 - val_accuracy: 0.6084
Epoch 3/1000
331/331 [==============================] - 0s 95us/step - loss: 0.6383 - accuracy: 0.6193 - val_loss: 0.6904 - val_accuracy: 0.5664
Epoch 4/1000
331/331 [==============================] - 0s 92us/step - loss: 0.6363 - accuracy: 0.5921 - val_loss: 0.6910 - val_accuracy: 0.5804
Epoch 5/1000
331/331 [==============================] - 0s 92us/step - loss: 0.6339 - accuracy: 0.6103 - val_loss: 0.6862 - val_accuracy: 0.5944
Epoch 6/1000
331/331 [==============================] - 0s 89us/step - loss: 0.6363 - accuracy: 0.6224 - val_loss: 0.7202 - val_accuracy: 0.5245
Epoch 7/1000
331/331 [==============================] - 0s 95us/step - loss: 0.674

331/331 [==============================] - 0s 123us/step - loss: 0.5645 - accuracy: 0.6979 - val_loss: 0.6642 - val_accuracy: 0.6434
Epoch 57/1000
331/331 [==============================] - 0s 137us/step - loss: 0.5673 - accuracy: 0.6979 - val_loss: 0.6246 - val_accuracy: 0.6573
Epoch 58/1000
331/331 [==============================] - 0s 119us/step - loss: 0.5720 - accuracy: 0.6918 - val_loss: 0.6930 - val_accuracy: 0.6573
Epoch 59/1000
331/331 [==============================] - 0s 103us/step - loss: 0.6007 - accuracy: 0.6798 - val_loss: 0.6210 - val_accuracy: 0.6434
Epoch 60/1000
331/331 [==============================] - 0s 97us/step - loss: 0.5923 - accuracy: 0.6737 - val_loss: 0.6102 - val_accuracy: 0.6434
Epoch 61/1000
331/331 [==============================] - 0s 108us/step - loss: 0.5877 - accuracy: 0.6767 - val_loss: 0.6350 - val_accuracy: 0.6434
Epoch 62/1000
331/331 [==============================] - 0s 99us/step - loss: 0.5900 - accuracy: 0.6798 - val_loss: 0.6377 - val_accu

Epoch 112/1000
331/331 [==============================] - 0s 79us/step - loss: 0.5068 - accuracy: 0.7492 - val_loss: 0.7101 - val_accuracy: 0.6434
Epoch 113/1000
331/331 [==============================] - 0s 87us/step - loss: 0.5485 - accuracy: 0.7311 - val_loss: 0.5734 - val_accuracy: 0.7063
Epoch 114/1000
331/331 [==============================] - 0s 89us/step - loss: 0.5111 - accuracy: 0.7372 - val_loss: 0.5664 - val_accuracy: 0.6853
Epoch 115/1000
331/331 [==============================] - 0s 74us/step - loss: 0.5101 - accuracy: 0.7402 - val_loss: 0.5928 - val_accuracy: 0.6923
Epoch 116/1000
331/331 [==============================] - 0s 84us/step - loss: 0.5067 - accuracy: 0.7492 - val_loss: 0.5476 - val_accuracy: 0.6853
Epoch 117/1000
331/331 [==============================] - 0s 78us/step - loss: 0.5013 - accuracy: 0.7432 - val_loss: 0.5321 - val_accuracy: 0.6853
Epoch 118/1000
331/331 [==============================] - 0s 79us/step - loss: 0.5174 - accuracy: 0.7523 - val_loss: 0

331/331 [==============================] - 0s 75us/step - loss: 0.4744 - accuracy: 0.7372 - val_loss: 0.5797 - val_accuracy: 0.7063
Epoch 168/1000
331/331 [==============================] - 0s 88us/step - loss: 0.4784 - accuracy: 0.7492 - val_loss: 0.5567 - val_accuracy: 0.6643
Epoch 169/1000
331/331 [==============================] - 0s 84us/step - loss: 0.5096 - accuracy: 0.7553 - val_loss: 0.5017 - val_accuracy: 0.7343
Epoch 170/1000
331/331 [==============================] - 0s 74us/step - loss: 0.4896 - accuracy: 0.7311 - val_loss: 0.5345 - val_accuracy: 0.6923
Epoch 171/1000
331/331 [==============================] - 0s 83us/step - loss: 0.4850 - accuracy: 0.7432 - val_loss: 0.5412 - val_accuracy: 0.7133
Epoch 172/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4644 - accuracy: 0.7613 - val_loss: 0.6315 - val_accuracy: 0.6783
Epoch 173/1000
331/331 [==============================] - 0s 73us/step - loss: 0.5742 - accuracy: 0.7160 - val_loss: 0.6618 - val_acc

331/331 [==============================] - 0s 74us/step - loss: 0.4714 - accuracy: 0.7432 - val_loss: 0.7113 - val_accuracy: 0.6993
Epoch 223/1000
331/331 [==============================] - 0s 83us/step - loss: 0.4786 - accuracy: 0.7432 - val_loss: 0.6359 - val_accuracy: 0.6993
Epoch 224/1000
331/331 [==============================] - 0s 88us/step - loss: 0.4532 - accuracy: 0.7613 - val_loss: 0.5928 - val_accuracy: 0.6993
Epoch 225/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4487 - accuracy: 0.7734 - val_loss: 0.5152 - val_accuracy: 0.6643
Epoch 226/1000
331/331 [==============================] - 0s 78us/step - loss: 0.4327 - accuracy: 0.7764 - val_loss: 0.6138 - val_accuracy: 0.6993
Epoch 227/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4290 - accuracy: 0.7553 - val_loss: 0.5597 - val_accuracy: 0.7273
Epoch 228/1000
331/331 [==============================] - 0s 85us/step - loss: 0.4309 - accuracy: 0.7855 - val_loss: 0.6908 - val_acc

...

Epoch 725/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4402 - accuracy: 0.7946 - val_loss: 0.6013 - val_accuracy: 0.7413
Epoch 726/1000
331/331 [==============================] - 0s 77us/step - loss: 0.4285 - accuracy: 0.7915 - val_loss: 0.4786 - val_accuracy: 0.7552
Epoch 727/1000
331/331 [==============================] - 0s 83us/step - loss: 0.4368 - accuracy: 0.7915 - val_loss: 0.5418 - val_accuracy: 0.7483
Epoch 728/1000
331/331 [==============================] - 0s 79us/step - loss: 0.4214 - accuracy: 0.7885 - val_loss: 0.4579 - val_accuracy: 0.7622
Epoch 729/1000
331/331 [==============================] - 0s 72us/step - loss: 0.4176 - accuracy: 0.7976 - val_loss: 0.4706 - val_accuracy: 0.7622
Epoch 730/1000
331/331 [==============================] - 0s 83us/step - loss: 0.5677 - accuracy: 0.7221 - val_loss: 0.5740 - val_accuracy: 0.6503
Epoch 731/1000
331/331 [==============================] - 0s 78us/step - loss: 0.4974 - accuracy: 0.7190 - val_loss: 0

Epoch 781/1000
331/331 [==============================] - 0s 87us/step - loss: 0.4095 - accuracy: 0.7915 - val_loss: 0.4650 - val_accuracy: 0.7483
Epoch 782/1000
331/331 [==============================] - 0s 80us/step - loss: 0.4039 - accuracy: 0.7825 - val_loss: 0.4484 - val_accuracy: 0.7552
Epoch 783/1000
331/331 [==============================] - 0s 77us/step - loss: 0.4194 - accuracy: 0.7674 - val_loss: 0.4502 - val_accuracy: 0.7552
Epoch 784/1000
331/331 [==============================] - 0s 87us/step - loss: 0.3871 - accuracy: 0.7915 - val_loss: 0.4442 - val_accuracy: 0.7343
Epoch 785/1000
331/331 [==============================] - 0s 71us/step - loss: 0.4008 - accuracy: 0.7915 - val_loss: 0.4592 - val_accuracy: 0.7552
Epoch 786/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3910 - accuracy: 0.8006 - val_loss: 0.4490 - val_accuracy: 0.7552
Epoch 787/1000
331/331 [==============================] - 0s 82us/step - loss: 0.4033 - accuracy: 0.7915 - val_loss: 0

Epoch 837/1000
331/331 [==============================] - 0s 79us/step - loss: 0.3763 - accuracy: 0.8066 - val_loss: 0.4488 - val_accuracy: 0.7552
Epoch 838/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3686 - accuracy: 0.8036 - val_loss: 0.5869 - val_accuracy: 0.7343
Epoch 839/1000
331/331 [==============================] - 0s 88us/step - loss: 0.3906 - accuracy: 0.7976 - val_loss: 0.4421 - val_accuracy: 0.7552
Epoch 840/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3753 - accuracy: 0.8036 - val_loss: 0.4671 - val_accuracy: 0.7692
Epoch 841/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3822 - accuracy: 0.8066 - val_loss: 0.5774 - val_accuracy: 0.7343
Epoch 842/1000
331/331 [==============================] - 0s 78us/step - loss: 0.4263 - accuracy: 0.7583 - val_loss: 0.4592 - val_accuracy: 0.7552
Epoch 843/1000
331/331 [==============================] - 0s 81us/step - loss: 0.3985 - accuracy: 0.7885 - val_loss: 0

Epoch 893/1000
331/331 [==============================] - 0s 89us/step - loss: 0.3683 - accuracy: 0.8157 - val_loss: 0.4427 - val_accuracy: 0.7832
Epoch 894/1000
331/331 [==============================] - 0s 103us/step - loss: 0.3613 - accuracy: 0.8097 - val_loss: 0.4873 - val_accuracy: 0.7552
Epoch 895/1000
331/331 [==============================] - 0s 106us/step - loss: 0.3610 - accuracy: 0.8157 - val_loss: 0.4379 - val_accuracy: 0.7552
Epoch 896/1000
331/331 [==============================] - 0s 105us/step - loss: 0.3726 - accuracy: 0.8097 - val_loss: 0.4382 - val_accuracy: 0.7552
Epoch 897/1000
331/331 [==============================] - 0s 99us/step - loss: 0.3651 - accuracy: 0.8097 - val_loss: 0.4522 - val_accuracy: 0.7692
Epoch 898/1000
331/331 [==============================] - 0s 99us/step - loss: 0.3616 - accuracy: 0.8066 - val_loss: 0.4303 - val_accuracy: 0.7762
Epoch 899/1000
331/331 [==============================] - 0s 98us/step - loss: 0.3630 - accuracy: 0.8036 - val_loss

331/331 [==============================] - 0s 70us/step - loss: 0.3461 - accuracy: 0.8127 - val_loss: 0.4649 - val_accuracy: 0.7692
Epoch 949/1000
331/331 [==============================] - 0s 79us/step - loss: 0.3433 - accuracy: 0.8187 - val_loss: 0.4389 - val_accuracy: 0.7622
Epoch 950/1000
331/331 [==============================] - 0s 78us/step - loss: 0.4138 - accuracy: 0.7553 - val_loss: 0.4256 - val_accuracy: 0.7832
Epoch 951/1000
331/331 [==============================] - 0s 97us/step - loss: 0.3526 - accuracy: 0.8278 - val_loss: 0.4329 - val_accuracy: 0.7692
Epoch 952/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3472 - accuracy: 0.8218 - val_loss: 0.4149 - val_accuracy: 0.7902
Epoch 953/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3465 - accuracy: 0.8248 - val_loss: 0.4267 - val_accuracy: 0.7552
Epoch 954/1000
331/331 [==============================] - 0s 95us/step - loss: 0.3495 - accuracy: 0.8157 - val_loss: 0.4407 - val_acc

In [131]:
acc_test_over3 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over3*100))

143/143 [==============================] - 0s 73us/step
combination test accuracy: 79.02%


In [132]:
############ Logistic Regression ############

In [133]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [134]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [135]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 98.32%


In [136]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [137]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 99.30%


In [138]:
############## Random Forest ##############

In [140]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [141]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 98.32%


In [142]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [143]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 100.00%


In [144]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [145]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         0.98181818 0.98181818 1.        ]
0.9927272727272728


In [146]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98507463 1.         0.98484848 0.98484848 0.98484848]
0.9879240162822253
